**Support code for Exploratory Report** <br>
https://datastudio.google.com/u/0/reporting/4cfc8b42-5e2a-4da5-8c03-e566264cefe4

In [1]:
import logging
import json
import pandas as pd
import pandas_gbq
from datetime import datetime
from web3 import Web3
from web3.exceptions import (ContractLogicError)
from eth_utils import (encode_hex, event_abi_to_log_topic)
from web3._utils.events import get_event_data
from google.oauth2 import service_account

In [2]:
abi =json.loads('''[{"name":"getReferenceDataBulk",
                      "inputs":[{"internalType":"string[]","name":"_bases","type":"string[]"},{"internalType":"string[]","name":"_quotes","type":"string[]"}],
                      "outputs":[{"components":[{"internalType":"uint256","name":"rate","type":"uint256"},{"internalType":"uint256","name":"lastUpdatedBase","type":"uint256"},{"internalType":"uint256","name":"lastUpdatedQuote","type":"uint256"}],"internalType":"struct IStdReference.ReferenceData[]","name":"","type":"tuple[]"}],
                      "stateMutability":"view",
                      "type":"function"
                      },
                      {"name":"RefDataUpdate",
                      "anonymous":false,
                      "inputs":[{"indexed":false,"internalType":"string","name":"symbol","type":"string"},{"indexed":false,"internalType":"uint64","name":"rate","type":"uint64"},{"indexed":false,"internalType":"uint64","name":"resolveTime","type":"uint64"},{"indexed":false,"internalType":"uint64","name":"requestId","type":"uint64"}],
                      "type":"event"
                      }
                    ]
                '''
               )

BandTokens = ['ALPHA', 'BAL', 'BAND', 'BNT', 'C98', 'COMP', 'CREAM', 'CRV', 'DAI', 'DPI', 'ETH', 'LINK', 'MATIC',
'MKR', 'REN', 'SNX', 'STRK', 'SUSD', 'SUSHI','UNI', 'USDC', 'USDT', 'WBTC', 'YFI']

BandAggregator = '0xDA7a001b254CD22e46d3eAB04d937489c93174C3'
#BandOracle = '0xA55d9ef16Af921b70Fed1421C1D298Ca5A3a18F1' # from block 14563636 onwards
BandOracleLegacy = '0xfc7A4c74beD0D761B9dC648F8730738D1449333a' #before block 14563636, 2022-04-11 09:35:26

# RPC must support archive mode, suggest using alchemy as it provide archive rpc for free
RPC_Endpoint = '' 
assert RPC_Endpoint, "Set up Archive Endpoint"

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())

Ethereum connected: True


In [3]:
def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]
def get_logs (w3, contract_address, topics, events, from_block=0, to_block=None):

    if not to_block:
        to_block = w3.eth.get_block('latest').number
    
    try:
        logs = w3.eth.get_logs({"address": contract_address
                               ,"topics":topics 
                               ,"fromBlock": from_block 
                               ,"toBlock": to_block
                               })
    except ValueError:
        logs = None
        from_block = 115*10**5
        batch_size =  5*10**4
        while from_block < to_block:
            print(datetime.utcnow(), f'from_block {from_block} to_block {from_block + batch_size}')
            batch_logs = w3.eth.get_logs({"address": contract_address
                                   ,"topics":topics 
                                   ,"fromBlock": from_block 
                                   ,"toBlock": from_block + batch_size # narrow down the range to avoid too many results error 
                                   })   
            if not logs:
                logs = batch_logs
            else:
                logs = logs + batch_logs
            if len(logs) > 0:    
                print(datetime.utcnow(), f'Total logs count {len(logs)}')  
            
            from_block = from_block + batch_size + 1
            
    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()            
            evt_abi = [x for x in events if encode_hex(event_abi_to_log_topic(x)) == evt_topic0][0]
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

blockTimestampCache = {}
def getBlockTimestamp(w3, blockNumber):
    if not blockNumber in blockTimestampCache:
        blockTimestampCache[blockNumber] = datetime.utcfromtimestamp(w3.eth.getBlock(blockNumber)['timestamp'])
    
    return blockTimestampCache[blockNumber]


In [4]:
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

bq_select = f'''with u as (select greatest(max(blockNumber), max(closest_cl_block)) as band_max_block from gearbox.band_price) 
               select 
                   distinct replace(cl.ticker,'btc-usd','wbtc-usd') as ticker, 
                   cl.blockNumber, cl.updated_at as block_timestamp, u.band_max_block
               from 
                   gearbox.oracle_price_history cl
                   left join u on 1=1
               where cl.ticker in ({','.join('"'+x.lower()+'-usd"' for x in BandTokens)})    
                   and cl.blockNumber > (select coalesce(band_max_block,0) from u)
               order by cl.blockNumber

            '''
df_cl = pandas_gbq.read_gbq(bq_select, 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
display(df_cl)


,ticker,blockNumber,block_timestamp,band_max_block
0,dai-usd,14727787.0,2022-05-07 04:45:05+00:00,14727694.0
1,comp-usd,14727799.0,2022-05-07 04:48:14+00:00,14727694.0
2,sushi-usd,14727804.0,2022-05-07 04:49:29+00:00,14727694.0
3,eth-usd,14727832.0,2022-05-07 04:55:27+00:00,14727694.0
4,link-usd,14727839.0,2022-05-07 04:56:17+00:00,14727694.0
...,...,...,...,...
2710,snx-usd,14757341.0,2022-05-11 21:53:38+00:00,14727694.0
2711,dpi-usd,14757342.0,2022-05-11 21:53:41+00:00,14727694.0
2712,uni-usd,14757342.0,2022-05-11 21:53:41+00:00,14727694.0
2713,comp-usd,14757342.0,2022-05-11 21:53:41+00:00,14727694.0


In [5]:
from_block = df_cl.loc[0,'band_max_block']
from_block = 0 if pd.isna(from_block) else int(from_block)+1
    
event_RefDataUpdate = get_event_abi(abi, 'RefDataUpdate')
topic_RefDataUpdate  = encode_hex(event_abi_to_log_topic(event_RefDataUpdate))
df_legacy = get_logs(w3_eth, BandOracleLegacy, [topic_RefDataUpdate], [event_RefDataUpdate], from_block)

if len(df_legacy)>0:
    df_legacy['symbol'] = df_legacy['args'].apply(lambda x: x['symbol'])
    df_legacy['ticker'] = df_legacy['symbol'].apply(lambda x: x.lower()+'-usd')
    df_legacy['rate'] = df_legacy['args'].apply(lambda x: x['rate']/1e9)
    df_legacy['tx_hash'] = df_legacy['transactionHash'].apply(lambda x: x.hex())

display(df_legacy)

""


In [6]:
df_legacy['closest_cl_block'] = None
for cnt, x in enumerate(df_legacy.itertuples()):
    cl = df_cl[(df_cl['ticker']==x.ticker)&(df_cl['blockNumber']>=x.blockNumber)&(df_cl['blockNumber']<x.blockNumber+10**4)]
    if len(cl) > 0:
        i = (cl['blockNumber'] - x.blockNumber).idxmin() 
        df_legacy.loc[(df_legacy['blockNumber']==x.blockNumber)&(df_legacy['ticker']==x.ticker),'closest_cl_block'] = cl.loc[i, 'blockNumber']
    if cnt%5000==0:
        print(datetime.utcnow(),cnt)
        
if len(df_legacy)>0:
    df_legacy = df_legacy[['blockNumber','closest_cl_block', 'symbol', 'ticker', 'rate', 'tx_hash']]
    legacy_tickers = list(df_legacy['ticker'].unique())
display(df_legacy)      

,closest_cl_block


In [7]:
if len(df_legacy)>0:
    number_cols = ['blockNumber','closest_cl_block', 'rate','deviation']
    for ticker in legacy_tickers:
        print(datetime.utcnow(), ticker, 'begin')
        df_legacy_ticker = df_legacy[df_legacy['ticker']==ticker].copy()
        df_legacy_ticker['deviation'] = df_legacy_ticker['rate']/df_legacy_ticker['rate'].shift(1) - 1

        df_legacy_ticker[number_cols] = df_legacy_ticker[number_cols].astype('float64')

        print(datetime.utcnow(), ticker, 'timestamp calc...')
        df_legacy_ticker['blockTimestamp'] = df_legacy_ticker['blockNumber'].apply(lambda x: getBlockTimestamp(w3_eth, int(x)))

        pandas_gbq.to_gbq(df_legacy_ticker, 
                          'gearbox.band_price',
                            project_id=gcp_project_id,
                            if_exists = ('replace' if legacy_tickers.index(ticker)==0 else 'append'), 
                            progress_bar = False)

In [8]:
bond_columns = ['blockNumber', 'blockTimestamp', 'closest_cl_block','symbol', 'ticker', 'rate', 'deviation', 'tx_hash']
number_cols = ['blockNumber','closest_cl_block', 'rate','deviation']
df_bond = pd.DataFrame(columns = bond_columns)
BandAggregator = Web3.toChecksumAddress(BandAggregator)

MaxLegacyBlock = df_legacy['blockNumber'].max() if len(df_legacy)>0 else 0

cl_blocks = df_cl[df_cl['blockNumber']>MaxLegacyBlock].groupby(['blockNumber','block_timestamp']).size().reset_index()

for cnt, b in enumerate(cl_blocks.itertuples()):
    try:
        ret = w3_eth.eth.contract(address=BandAggregator, abi=abi).functions.getReferenceDataBulk(BandTokens,
                                                                                        ['USD' for x in BandTokens]
                                                                                       ).call(block_identifier = int(b.blockNumber))
        for i, t in enumerate(ret):
            df_bond.loc[len(df_bond),:] = [b.blockNumber,
                                         b.block_timestamp,      
                                         b.blockNumber,
                                         BandTokens[i],
                                         BandTokens[i].lower()+'-usd',
                                         t[0]/1e18,
                                         None,
                                         None]
    except ContractLogicError as err:
        logging.error(block, str(int(blockNumber)))
        logging.error(err)
    
    if cnt%500==0 or cnt==len(cl_blocks):
        print(datetime.utcnow(),f'{cnt}/{len(cl_blocks)}..')        


tickers = list(df_bond['ticker'].unique())
for ticker in tickers:
    df_bond_ticker = df_bond[df_bond['ticker']==ticker].copy()
    df_bond_ticker['deviation'] = df_bond_ticker['rate']/df_bond_ticker['rate'].shift(1) - 1
    df_bond_ticker[number_cols] = df_bond_ticker[number_cols].astype('float64')
    df_bond_ticker['blockTimestamp'] = df_bond_ticker['blockTimestamp'].astype('datetime64')
    pandas_gbq.to_gbq(df_bond_ticker, 
                      'gearbox.band_price',
                      project_id=gcp_project_id,
                      if_exists = 'append', 
                      progress_bar = False)
            

2022-05-12 01:27:04.030740 0/2272..
2022-05-12 01:29:53.841345 500/2272..
2022-05-12 01:32:56.909889 1000/2272..
2022-05-12 01:36:26.139411 1500/2272..
2022-05-12 01:40:13.695923 2000/2272..


**Exploratory Report** <br>
https://datastudio.google.com/u/0/reporting/4cfc8b42-5e2a-4da5-8c03-e566264cefe4